# **imort libraries**

***导入第三方库***

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import math
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable

import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as data
import torchvision
from torchvision import transforms

from keras.layers import Input, Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten
from keras.models import Model
import matplotlib.pyplot as plt
import pickle

**Import the data**

***对数据进行导入***

In [2]:
PATH = os.path.join('/home/mw/input/', 'dataset')

**The data set has the following directory structure:**

***数据集具有以下目录结构：***
```dataset
|__ train
    |______ real: []
    |______ fake: []
|__ test
    |______ real: []
    |______ fake: []
```
**After extracting its content, assign appropriate file paths to the variables for training and validation sets.**

***提取其内容后，为变量分配适当的文件路径以用于训练和验证集。***


In [3]:
train_dir = os.path.join(PATH, 'train')
test_dir = os.path.join(PATH, 'validation')

train_real_dir = os.path.join(train_dir, 'real') 
train_fake_dir = os.path.join(train_dir, 'fake')  
test_real_dir = os.path.join(test_dir, 'real')  
test_fake_dir = os.path.join(test_dir, 'fake')

# **get to know the dataset**

***了解数据集***

**Let's see how many real and fake images are in the training and validation catalog:**

***让我们看看训练和验证目录中有多少真实和虚假的图像：***

In [4]:
num_real_tr = len(os.listdir(train_real_dir))
num_fake_tr = len(os.listdir(train_fake_dir))

num_real_test = len(os.listdir(test_real_dir))
num_fake_test = len(os.listdir(test_fake_dir))

total_train = num_real_tr + num_fake_tr
total_test = num_real_test + num_fake_test

print('total training real images:', num_real_tr)
print('total training fake images:', num_fake_tr)

print('total test real images:', num_real_test)
print('total test fake images:', num_fake_test)
print("--")
print("Total training images:", total_train)
print("Total test images:", total_test)

total training real images: 3613
total training fake images: 5866
total test real images: 1548
total test fake images: 2513
--
Total training images: 9479
Total test images: 4061


# 资料准备
将图像格式化为经过适当预处理的浮点张量，然后再馈入网络：

1. 从磁盘读取图像
2. 解码这些图像的内同，并根据其RGB内容将其转换为正确的网格格式
3. 将它们转换为浮点张量
4. 将张量从0到255之间的值重新缩放为0到1之间的值，因为神经网络更喜欢处理较小的输入值


解码这些图像的内容，并根据其RGB内容将其转换为正确的网格格式。
将它们转换为浮点张量。
将张量从0到255之间的值重新缩放为0到1之间的值，因为神经网络更喜欢处理较小的输入值。
幸运的是，所有这些任务都可以通过提供的ImageDataGenerator类来完成tf.keras。它可以从磁盘读取图像并将其预处理为适当的张量。它还将设置将这些图像转换成张量的生成器，这对于训练网络很有帮助。

训练量很少时，通常会发生过度拟合。解决此问题的一种方法是扩充数据集，使其具有足够数量的训练示例。数据增强采用通过使用产生真实感图像的随机变换增强样本来从现有训练样本生成更多训练数据的方法。目标是模型在训练期间永远不会看到两次完全相同的图片。这有助于使模型暴露于数据的更多方面，并且可以更好地进行概括。

tf.keras使用ImageDataGenerator类来实现这一点。将不同的转换传递给数据集，它将在训练过程中加以应用。
对训练图像进行重新缩放，45度旋转，宽度偏移，高度偏移，水平翻转和缩放增强。防止出现过拟合的情况

In [5]:
BATCH_SIZE = 128
transform_method = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train_data = torchvision.datasets.ImageFolder(root=train_dir, transform=transform_method)
train_data_loader = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2)
test_data = torchvision.datasets.ImageFolder(root=test_dir, transform=transform_method)
test_data_loader  = data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2) 

trainset = train_data
trainloader = train_data_loader
testset = test_data
testloader = test_data_loader

In [6]:
print("Number of train samples: ", len(train_data))
print("Number of test samples: ", len(test_data))
print("Detected Classes are: ", train_data.class_to_idx) # classes are detected by folder structure
print("num classes = ", len(train_data.class_to_idx))

Number of train samples:  9479
Number of test samples:  4061
Detected Classes are:  {'fake': 0, 'real': 1}
num classes =  2


# 构建神经网络
减少过度拟合的另一种技术是将丢失引入网络。这是一种正则化形式，它迫使网络中的权重仅取较小的值，这使得权重值的分配更加规则，并且网络可以减少在小的训练样本上的过度拟合。

当您在图层上应用滤除时，它会在训练过程中从所应用的图层中随机滤除（设置为零）数量的输出单位。dropout采用分数形式作为其输入值，形式为0.1、0.2、0.4等。这意味着从所施加的层中随机退出输出单元的10％，20％或40％。将0.1的dropout应用于某个图层时，它会在每个训练时期随机杀死10％的输出单位。使用此新的dropout功能创建网络体系结构，并将其应用于不同的卷积和完全连接的层。

In [7]:
class Net3(nn.Module):
    def __init__(self):
        super(Net3, self).__init__()
        self.conv1 = nn.Conv2d(3, 28, 3,padding=1)
        self.bn1 = nn.BatchNorm2d(28)
        self.conv2 = nn.Conv2d(28, 35, 3,padding=1)
        self.bn2 = nn.BatchNorm2d(35)
        self.pool = nn.MaxPool2d(2, 2)
        self.drop1 = nn.Dropout(p=0.2)
        self.conv3 = nn.Conv2d(35, 16, 3,padding=1)
        self.bn3 = nn.BatchNorm2d(16)
        
        self.fc1 = nn.Linear(16 * 8 * 8, 100)
        self.drop2 = nn.Dropout(p=0.3)
        self.fc2 = nn.Linear(100, 30)
        self.bn4 = nn.BatchNorm1d(30)
        self.fc3 = nn.Linear(30, 10)

    def forward(self, x):
        x = self.bn1(F.relu(self.conv1(x)))
        x = self.drop1(x)
        x = self.bn2(F.relu(self.conv2(x)))
        x = self.pool(x)
        x = self.bn3(F.relu(self.conv3(x)))
        x = self.pool(x)
        x = x.view(-1, 16 * 8 * 8)
        x = self.drop1(x)
        x = F.relu(self.fc1(x))
        x = self.drop2(x)
        x = self.bn4(F.relu(self.fc2(x)))
        x = self.fc3(x)
    
        return F.log_softmax(x, dim=0)


# Plot with Test Error

In [8]:
def compute_test_acc(model, testloader):
    # Test the model
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        count = 0
        for images, labels in testloader:
            count += 1
            images = Variable(images)
            labels = Variable(labels)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return (correct / total)

def compute_train_acc(model, trainloader):
    # Test the model
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        count = 0
        for images, labels in trainloader:
            count += 1
            images = Variable(images)
            labels = Variable(labels)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return (correct / total)

In [9]:
# Find optimal_batch
def uncertainty_metric_max_min_difference_selection(model, trainloader):
    batch_evaluation = []
    for i, (images, labels) in enumerate(trainloader):
        outputs = model(images)
        batch_differences = []
        for j in range(len(outputs)):
            min_prob = min(outputs[j])
            max_prob = max(outputs[j])
            diff = float(max_prob - min_prob)
            batch_differences.append(diff)
        batch_mean = np.mean(batch_differences)
        batch_evaluation.append(batch_mean)
    batch_evaluation = np.array(batch_evaluation)
    k = 20
#     largest = list((-batch_evaluation).argsort()[:k])
    smallest = list(np.argsort(batch_evaluation)[:k])
    return smallest

In [10]:
loss_list = []
acc_list = []
train_err_list = []
test_err_list = []

for trial in range(1):
    active_model = Net3()
    free_params = sum(p.numel() for p in active_model.parameters() if p.requires_grad)
    print(free_params)

    trainstep = 125
    # Loss and optimizer

    criterion = nn.NLLLoss() #You can modify the loss function
    optimizer = optim.SGD(active_model.parameters(), lr=0.005, momentum=0.9, weight_decay=8e-4) #You can change the optimizer

    # Train the model

    total_step = len(trainloader)
    print(total_step)
    trial_loss_list = []
    trial_acc_list = []

    trial_train_err_list = []
    trial_test_err_list = []

    num_epochs = 450

    for epoch in range(num_epochs):
        print("epoch: ", epoch)
        total = 0
        correct = 0
        randomly_selected = uncertainty_metric_max_min_difference_selection(active_model, trainloader)
        for i, (images, labels) in enumerate(trainloader):
            if i not in randomly_selected:
                continue

    #         images = images.cuda(async=True)
    #         labels = labels.cuda(async=True)

            images = Variable(images)
            labels = Variable(labels)
            # Run the forward pass
            # print("running forward pass....")
            outputs = active_model(images)

            loss = criterion(outputs, labels)
            trial_loss_list.append(loss.item())

            # Backprop 
            # print("backpropagating......")
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

             # Track the accuracy
            total = labels.size(0) + total
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item() + correct
            trial_acc_list.append(correct / total)


            if (i + 1) % trainstep == 0:
                w = torch.nn.utils.parameters_to_vector(active_model.parameters())
                print(w)
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                      .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                              (correct / total) * 100))
        trial_train_err_list.append(compute_train_acc(active_model, trainloader))
        trial_test_err_list.append(compute_test_acc(active_model, testloader))
        if (total == correct):
            break 

#     loss_list.append(np.array(trial_loss_list))
#     acc_list.append(trial_acc_list)
    train_err_list.append(trial_train_err_list)
    test_err_list.append(trial_test_err_list)
            
print('Finished Training') 

# loss_list = np.array(loss_list)
# acc_list = np.array(acc_list)
train_err_list= np.array(train_err_list)
test_err_list=np.array(test_err_list)

# loss_list = np.mean(loss_list, axis=0)
# acc_list = np.mean(acc_list, axis=0)
train_err_list= np.mean(train_err_list, axis=0)
test_err_list=np.mean(test_err_list, axis=0)




120753
75
epoch:  0
epoch:  1
epoch:  2
epoch:  3
epoch:  4
epoch:  5
epoch:  6
epoch:  7
epoch:  8
epoch:  9
epoch:  10
epoch:  11
epoch:  12
epoch:  13
epoch:  14
epoch:  15
epoch:  16
epoch:  17
epoch:  18
epoch:  19
epoch:  20
epoch:  21
epoch:  22
epoch:  23
epoch:  24
epoch:  25
epoch:  26
epoch:  27
epoch:  28
epoch:  29
epoch:  30
epoch:  31
epoch:  32
epoch:  33
epoch:  34
epoch:  35
epoch:  36
epoch:  37
epoch:  38
epoch:  39
epoch:  40
epoch:  41
epoch:  42
epoch:  43
epoch:  44
epoch:  45
epoch:  46
epoch:  47
epoch:  48
epoch:  49
epoch:  50
epoch:  51
epoch:  52
epoch:  53
epoch:  54
epoch:  55
epoch:  56
epoch:  57
epoch:  58
epoch:  59
epoch:  60
epoch:  61
epoch:  62
epoch:  63
epoch:  64
epoch:  65
epoch:  66
epoch:  67
epoch:  68
epoch:  69
epoch:  70
epoch:  71
epoch:  72
epoch:  73
epoch:  74
epoch:  75
epoch:  76
epoch:  77
epoch:  78
epoch:  79
epoch:  80
epoch:  81
epoch:  82
epoch:  83
epoch:  84
epoch:  85
epoch:  86
epoch:  87
epoch:  88
epoch:  89
epoch:  90

In [11]:
# testing = []
# testing.append([1,2,3])
# testing.append([4,2,3])
# testing = np.array(testing)
# print(np.mean(testing, axis=0))

In [13]:
# ... after training, save your model 
model_filename = '/home/mw/work/active_model3_minimum.pt'
torch.save(active_model.state_dict(), model_filename)

# .. to load your previously training model:
# model2 = Net3()
# model2.load_state_dict(torch.load(model_filename))
# model2.eval()

In [14]:
active_train_err_list = train_err_list
active_test_err_list = test_err_list
active_acc_list1 = acc_list



In [16]:
pickle_out = open("/home/mw/work/test1_largest_margin_minimum_training_accuracy.pickle","wb")
pickle.dump(active_train_err_list, pickle_out)
pickle_out.close()


pickle_out = open("/home/mw/work/test1_largest_margin_minimum_testing_accuracy.pickle","wb")
pickle.dump(active_test_err_list, pickle_out)
pickle_out.close()

# pickle_out = open("test1_random_testing_accuracy.pickle","wb")
# pickle.dump(new_rand_test_err_list, pickle_out)
# pickle_out.close()

# Random Model

In [17]:
# random_model2 = Net3()

In [18]:
rand_loss_list = []
rand_acc_list = []
rand_train_err_list = []
rand_test_err_list = []

for trial in range(1):
    random_model2 = Net3()
    free_params = sum(p.numel() for p in random_model2.parameters() if p.requires_grad)

    trainstep = 125
    # Loss and optimizer

    criterion = nn.NLLLoss() #You can modify the loss function
    optimizer = optim.SGD(random_model2.parameters(), lr=0.005, momentum=0.9, weight_decay=8e-4) #You can change the optimizer

    # Train the model

    total_step = len(trainloader)
    print(total_step)
    trial_rand_loss_list = []
    trial_rand_acc_list = []

    trial_rand_train_err_list = []
    trial_rand_test_err_list = []

    num_epochs = 450

    for epoch in range(num_epochs):
        print("epoch: ", epoch)
        total = 0
        correct = 0
        randomly_selected = np.random.choice(range(len(list(trainloader))), size=20)
        for i, (images, labels) in enumerate(trainloader):
            if i not in randomly_selected:
                continue

    #         images = images.cuda(async=True)
    #         labels = labels.cuda(async=True)

            images = Variable(images)
            labels = Variable(labels)
            # Run the forward pass
            # print("running forward pass....")
            outputs = random_model2(images)

            loss = criterion(outputs, labels)
            rand_loss_list.append(loss.item())

            # Backprop 
            # print("backpropagating......")
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

             # Track the accuracy
            total = labels.size(0) + total
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item() + correct
            rand_acc_list.append(correct / total)


            if (i + 1) % trainstep == 0:
                w = torch.nn.utils.parameters_to_vector(random_model2.parameters())
                print(w)
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                      .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                              (correct / total) * 100))
        trial_rand_train_err_list.append(compute_train_acc(random_model2, trainloader))
        trial_rand_test_err_list.append(compute_test_acc(random_model2, testloader))
        if (total == correct):
            break 

    rand_loss_list.append(trial_rand_loss_list)
    rand_acc_list.append(trial_rand_acc_list)
    rand_train_err_list.append(trial_rand_train_err_list)
    rand_test_err_list.append(trial_rand_test_err_list)
            
print('Finished Training') 

# rand_loss_list = np.array(rand_loss_list)
# rand_acc_list = np.array(rand_acc_list)
rand_train_err_list= np.array(rand_train_err_list)
rand_test_err_list=np.array(rand_test_err_list)

# rand_loss_list = np.mean(rand_loss_list, axis=0)
# rand_acc_list = np.mean(rand_acc_list, axis=0)
rand_train_err_list= np.mean(rand_train_err_list, axis=0)
rand_test_err_list=np.mean(rand_test_err_list, axis=0)





75
epoch:  0
epoch:  1
epoch:  2
epoch:  3
epoch:  4
epoch:  5
epoch:  6
epoch:  7
epoch:  8
epoch:  9
epoch:  10
epoch:  11
epoch:  12
epoch:  13
epoch:  14
epoch:  15
epoch:  16
epoch:  17
epoch:  18
epoch:  19
epoch:  20
epoch:  21
epoch:  22
epoch:  23
epoch:  24
epoch:  25
epoch:  26
epoch:  27
epoch:  28
epoch:  29
epoch:  30
epoch:  31
epoch:  32
epoch:  33
epoch:  34
epoch:  35
epoch:  36
epoch:  37
epoch:  38
epoch:  39
epoch:  40
epoch:  41
epoch:  42
epoch:  43
epoch:  44
epoch:  45
epoch:  46
epoch:  47
epoch:  48
epoch:  49
epoch:  50
epoch:  51
epoch:  52
epoch:  53
epoch:  54
epoch:  55
epoch:  56
epoch:  57
epoch:  58
epoch:  59
epoch:  60
epoch:  61
epoch:  62
epoch:  63
epoch:  64
epoch:  65
epoch:  66
epoch:  67
epoch:  68
epoch:  69
epoch:  70
epoch:  71
epoch:  72
epoch:  73
epoch:  74
epoch:  75
epoch:  76
epoch:  77
epoch:  78
epoch:  79
epoch:  80
epoch:  81
epoch:  82
epoch:  83
epoch:  84
epoch:  85
epoch:  86
epoch:  87
epoch:  88
epoch:  89
epoch:  90
epoch:

## Compare Largest Margin to Random

In [36]:
new_active_train_err_list = [0]
new_active_train_err_list.extend(active_train_err_list)
# plt.plot(new_active_train_err_list)
# plt.plot(active_test_err_list)
new_rand_train_err_list = [0]
new_rand_train_err_list.extend(rand_train_err_list)
# plt.plot(new_rand_train_err_list)
# # plt.plot(active_test_err_list)
# plt.legend(['active', 'random'])
# plt.title("Active Training Set Accuracy")
# plt.xlabel("Epoch")
# plt.ylabel("Training Set Accuracy")
# plt.savefig("/home/mw/work/test3_4_acc1.png")
# plt.show()

pickle_out = open("/home/mw/work/test1_largest_margin_minimum2_training_accuracy.pickle","wb")
pickle.dump(new_active_train_err_list, pickle_out)
pickle_out.close()

pickle_out = open("/home/mw/work/test1_random2_training_accuracy.pickle","wb")
pickle.dump(new_rand_train_err_list, pickle_out)
pickle_out.close()

In [37]:
new_active_test_err_list = [0]
new_active_test_err_list.extend(active_test_err_list)

# plt.plot(new_second_train_err_list)
# plt.plot(active_test_err_list)
new_rand_test_err_list = [0]
new_rand_test_err_list.extend(rand_test_err_list)

# plt.plot(new_active_test_err_list)
# # plt.plot(active_test_err_list)
# plt.plot(new_rand_test_err_list)
# # plt.plot(active_test_err_list)
# plt.legend(['active', 'random'])
# plt.title("Active Test Set Accuracy")
# plt.xlabel("Epoch")
# plt.ylabel("Test Set Accuracy")
# plt.savefig("/home/mw/work/test3_4_acc2.png")
# plt.show()

pickle_out = open("/home/mw/work/test2_largest_margin_minimum2_testing_accuracy.pickle","wb")
pickle.dump(new_active_test_err_list, pickle_out)
pickle_out.close()

pickle_out = open("/home/mw/work/test2_random2_testing_accuracy.pickle","wb")
pickle.dump(new_rand_test_err_list, pickle_out)
pickle_out.close()

In [21]:
# ... after training, save your model 
model_filename = '/home/mw/work/random_model2.pt'
torch.save(random_model2.state_dict(), model_filename)

# .. to load your previously training model:
# model2 = Net3()
# model2.load_state_dict(torch.load(model_filename))
# model2.eval()

In [38]:
plt.plot(active_train_err_list)
# plt.plot(active_test_err_list)
plt.plot(rand_train_err_list)
# plt.plot(active_test_err_list)
plt.legend(['active', 'random'])
plt.title("Active Training Set Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Training Set Accuracy")
plt.savefig("/home/mw/work/test2_acc1_min.png")
plt.show()

<Figure size 432x288 with 1 Axes>